# Analysis of variance

1因子の場合

In [16]:
import numpy as np
import pandas as pd

In [2]:
#  データの作成
data = np.array([
    [33,31,33],
    [30,29,31],
    [33,28,32],
    [29,29,32],
    [32,27,36]])
df = pd.DataFrame(data,columns=['Mimizu','Batta','Mix'],index=[1,2,3,4,5])
df

,Mimizu,Batta,Mix
1,33,31,33
2,30,29,31
3,33,28,32
4,29,29,32
5,32,27,36


In [3]:
# 全データの平均
all_mean = df.stack().mean()
all_mean

31.0

In [4]:
# 列の効果
df_effect = df.mean(axis=0) - all_mean
df_effect

Mimizu    0.4
Batta    -2.2
Mix       1.8
dtype: float64

In [5]:
# 誤差 - データから列の平均を引く
df_error = df - df.mean()
df_error

,Mimizu,Batta,Mix
1,1.6,2.2,0.2
2,-1.4,0.2,-1.8
3,1.6,-0.8,-0.8
4,-2.4,0.2,-0.8
5,0.6,-1.8,3.2


In [6]:
# 誤差の合計は0になる
df_error.sum()

Mimizu    7.105427e-15
Batta    -3.552714e-15
Mix       1.421085e-14
dtype: float64

In [18]:
# ---- numpy のみで計算するバージョン
data = np.array([
    [33,31,33],
    [30,29,31],
    [33,28,32],
    [29,29,32],
    [32,27,36]])

all_mean = np.full(data.shape, data.mean())
col_effect = data.mean(axis=0) - all_mean
err = data - all_mean - col_effect

print(all_mean)
print(col_effect)
print(err)

[[31. 31. 31.]
 [31. 31. 31.]
 [31. 31. 31.]
 [31. 31. 31.]
 [31. 31. 31.]]
[[ 0.4 -2.2  1.8]
 [ 0.4 -2.2  1.8]
 [ 0.4 -2.2  1.8]
 [ 0.4 -2.2  1.8]
 [ 0.4 -2.2  1.8]]
[[ 1.6  2.2  0.2]
 [-1.4  0.2 -1.8]
 [ 1.6 -0.8 -0.8]
 [-2.4  0.2 -0.8]
 [ 0.6 -1.8  3.2]]


不偏分散の式
$$
V = \frac{\sum(x_i - \bar{x})^2}{n-1}
$$

In [7]:
# 列の効果の不偏分散 V1

dfn = df.columns.size - df_effect.mean().size #分子の自由度
V_1 = np.sum(df.index.size * (np.square(df_effect - df_effect.mean() ))) / dfn
V_1

20.59999999999996

In [8]:
# 誤差の不偏分散 V2

err = df_error.stack()
dfd = err.size - df_error.mean().size #分母の自由度
V_2 = np.square(err - err.mean()).sum() / dfd
V_2

3.0666666666666664

In [9]:
# F値 V1/V2
F = V_1 / V_2
F

6.717391304347813

In [10]:
from scipy.stats import f

five = f.ppf(0.95, dfn,dfd)
one = f.ppf(0.99, dfn, dfd)
print('上側確率 5%',five)
print('上側確率 1%',one)

上側確率 5% 3.8852938346523933
上側確率 1% 6.9266081401913


# Analysis of Variance
2因子の場合

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import f

In [2]:
data = np.array([
    [29,35,29,29],
    [26,33,30,28],
    [32,34,33,34]
    ])
df = pd.DataFrame(data, columns=['火','砂','腐','粘'], index=['ミ','バ','混'])
df

,火,砂,腐,粘
ミ,29,35,29,29
バ,26,33,30,28
混,32,34,33,34


In [9]:
df.stack().mean()
df.mean(axis=1)

ミ    30.50
バ    29.25
混    33.25
dtype: float64

In [3]:
# 行の効果（餌の効果）行の平均-全体平均
row_effect = df.mean(axis=1) - df.stack().mean()
row_effect

ミ   -0.50
バ   -1.75
混    2.25
dtype: float64

In [11]:
# 列の効果（土の効果） 列の平均 - 全体平均
col_effect = df.mean(axis=0) - df.stack().mean()
col_effect

火   -2.000000
砂    3.000000
腐   -0.333333
粘   -0.666667
dtype: float64

In [12]:
# 誤差 データの値 - 全体平均　- 行の効果 - 列の効果
# 本来は df - df.stack().mean() - row_effect - col_effect だが、うまく計算のできないので分割して計算する

df_error = df - df.stack().mean()
df_error

,火,砂,腐,粘
ミ,-2.0,4.0,-2.0,-2.0
バ,-5.0,2.0,-1.0,-3.0
混,1.0,3.0,2.0,3.0


In [30]:
# 誤差から行の効果と列の効果を分離する
df_error = df_error.subtract(row_effect, axis=0)
df_error = df_error.subtract(col_effect, axis=1)
df_error

,火,砂,腐,粘
ミ,0.50,1.50,-1.166667,-0.833333
バ,-1.25,0.75,1.083333,-0.583333
混,0.75,-2.25,0.083333,1.416667


In [40]:
# 誤差の不偏分散
phi_2 = (df.columns.size - 1) * (df.index.size -1)
V_2 = np.sum(np.square(df_error.stack())) / phi_2
V_2

2.638888888888889

In [45]:
# 行の不偏分散
phi_11 = row_effect.size - df.stack().mean().size
V_11 = np.sum(np.square(row_effect) * df.columns.size) / phi_11
V_11

16.75

In [48]:
# F1 行の効果の有意差
F_1 = V_11 / V_2

five = f.ppf(0.95, phi_11,phi_2)
one = f.ppf(0.99, phi_11,phi_2)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('F1 = ',F_1)

上側確率 5% 5.143252849784718
上側確率 1% 10.92476650083833
F =  6.347368421052631


In [49]:
# 列の不偏分散
phi_12 = col_effect.size - df.stack().mean().size
V_12 = np.sum(np.square(col_effect) * df.index.size) / phi_12
V_12

13.555555555555555

In [52]:
# F2 列の効果の有意差

F_2 = V_12 / V_2

five = f.ppf(0.95, phi_12,phi_2)
one = f.ppf(0.99, phi_12,phi_2)
print('上側確率 5%',five)
print('上側確率 1%',one)
print('F2 = ',F_2)

上側確率 5% 4.757062663089414
上側確率 1% 9.779538240923273
F =  5.136842105263158


In [15]:
# ---- numpy のみで 計算する場合

data = np.array([
    [29,35,29,29],
    [26,33,30,28],
    [32,34,33,34]
    ])

all_mean = np.full(data.shape, data.mean())
col_effect = data.mean(axis=0) - all_mean
row_effect = data.mean(axis=1)[:, np.newaxis] - all_mean
err = data - all_mean - col_effect - row_effect

print(all_mean)
print(col_effect)
print(row_effect)
print(err)
# ----

[[31. 31. 31. 31.]
 [31. 31. 31. 31.]
 [31. 31. 31. 31.]]
[[-2.          3.         -0.33333333 -0.66666667]
 [-2.          3.         -0.33333333 -0.66666667]
 [-2.          3.         -0.33333333 -0.66666667]]
[[-0.5  -0.5  -0.5  -0.5 ]
 [-1.75 -1.75 -1.75 -1.75]
 [ 2.25  2.25  2.25  2.25]]
[[ 0.5         1.5        -1.16666667 -0.83333333]
 [-1.25        0.75        1.08333333 -0.58333333]
 [ 0.75       -2.25        0.08333333  1.41666667]]


# Analysis of Variance
3因子の場合 ラテン方格

In [2]:
import numpy as np

In [46]:
data = np.array([
    [31,33,31,33],
    [34,24,29,29],
    [35,32,30,39],
    [30,25,34,27]])

In [124]:
def factor_matrix(factor) -> np.ndarray:
    num = len(factor)
    mat =[]
    for i in range(0,num):
        for j in range(0,num):
            index = (i + j) % num
            mat.append(factor[index])
    return np.array(mat).reshape(num,num)

factor_mat = factor_matrix(['A','B','C','D'])
print(factor_mat)

[['A' 'B' 'C' 'D']
 ['B' 'C' 'D' 'A']
 ['C' 'D' 'A' 'B']
 ['D' 'A' 'B' 'C']]


In [48]:
# 全体平均
all_mean = data.mean()
print(all_mean)

31.0


In [106]:
# 列の効果
col_effect = data.mean(axis=0) - all_mean
col_effect = np.vstack([col_effect] * 4)
print(col_effect)

# 行の効果
row_effect = data.mean(axis=1) - all_mean
row_effect = np.hstack([row_effect[:,np.newaxis]] * 4)
print(row_effect)

[[ 1.5 -2.5  0.   1. ]
 [ 1.5 -2.5  0.   1. ]
 [ 1.5 -2.5  0.   1. ]
 [ 1.5 -2.5  0.   1. ]]
[[ 1.  1.  1.  1.]
 [-2. -2. -2. -2.]
 [ 3.  3.  3.  3.]
 [-2. -2. -2. -2.]]


In [54]:
# A, B, C, D それぞれの効果

a_data = (factor_mat == 'A') * data
a_effect = np.mean(a_data[a_data > 0]) - all_mean
a_effect *= (factor_mat == 'A')

b_data = (factor_mat == 'B') * data
b_effect = np.mean(b_data[b_data > 0]) - all_mean
b_effect *= (factor_mat == 'B')

c_data = (factor_mat == 'C') * data
c_effect = np.mean(c_data[c_data > 0]) - all_mean
c_effect *= (factor_mat == 'C')

d_data = (factor_mat == 'D') * data
d_effect = np.mean(d_data[d_data > 0]) - all_mean
d_effect *= (factor_mat == 'D')

print(a_effect)
print(b_effect)
print(c_effect)
print(d_effect)

[[-2.25 -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -2.25]
 [-0.   -0.   -2.25 -0.  ]
 [-0.   -2.25 -0.   -0.  ]]
[[0. 4. 0. 0.]
 [4. 0. 0. 0.]
 [0. 0. 0. 4.]
 [0. 0. 4. 0.]]
[[-0.   -0.   -1.75 -0.  ]
 [-0.   -1.75 -0.   -0.  ]
 [-1.75 -0.   -0.   -0.  ]
 [-0.   -0.   -0.   -1.75]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [123]:
# 誤差の計算
error = data - all_mean - col_effect - row_effect - a_effect - b_effect - c_effect
print(error)

[[-0.25 -0.5   0.75  0.  ]
 [-0.5  -0.75  0.    1.25]
 [ 1.25  0.5  -1.75  0.  ]
 [-0.5   0.75  1.   -1.25]]


In [113]:
# 誤差の不偏分散
phi_2 = 6.0
V_2 = np.sum(np.square(error)) / phi_2

In [114]:
# row
phi_11 = 3.0
V_11 = np.sum(np.square(row_effect)) / phi_11
F_1 = V_11 / V_2

In [115]:
# col
phi_12 = 3.0
V_12 = np.sum(np.square(col_effect)) / phi_12
F_2 = V_12 / V_2

In [116]:
# factor 3
phi_13 = 3.0
V_13 = (np.sum(np.square(a_effect)) + np.sum(np.square(b_effect)) + np.sum(np.square(c_effect)) + np.sum(np.square(d_effect))) / phi_13
F_3 = V_13 / V_2

In [118]:
from scipy.stats import f

In [122]:
five = f.ppf(0.95, phi_11,phi_2)
one = f.ppf(0.99, phi_11,phi_2)
print('上側確率 5%',five)
print('上側確率 1%',one)
print(f'F_1 = {F_1}')
print(f'F_2 = {F_2}')
print(f'F_3 = {F_3}')

上側確率 5% 4.757062663089414
上側確率 1% 9.779538240923273
F_1 = 12.521739130434781
F_2 = 6.608695652173912
F_3 = 16.782608695652172


# 相互作用

In [1]:
import numpy as np

In [2]:
d1 = np.array([
        [40,36],
        [33,25]])
d2 = np.array([
        [33,30],
        [28,23]])

In [3]:
data = np.array([[
        [40,36],
        [33,25]],
        [[33,30],
        [28,23]]])
print(data)

[[[40 36]
  [33 25]]

 [[33 30]
  [28 23]]]


In [4]:
# S_axb

mat1 = np.array([[
        [1,0],
        [0,1]],
        [[1,0],
        [0,1]]])
i1 = (np.sum(data * mat1) / 4.0) - 31.0
print(i1)

mat2 = -mat1 + 1
i2 = (np.sum(data * mat2) / 4.0) - 31.0
print(i2)

Sab = 4 * np.square(i1) + 4 * np.square(i2)
print(Sab)

-0.75
0.75
4.5


In [5]:
# S_axc

mat1 = np.array([[
        [1,1],
        [0,0]],
        [[0,0],
        [1,1]]])
i1 = (np.sum(data * mat1) / 4.0) - 31.0
print(i1)

mat2 = -mat1 + 1
i2 = (np.sum(data * mat2) / 4.0) - 31.0
print(i2)

Sac = 4 * np.square(i1) + 4 * np.square(i2)
print(Sac)

0.75
-0.75
4.5


In [8]:
# S_bxc
mat1 = np.array([[
        [1,0],
        [1,0]],[
        [0,1],
        [0,1]]])
i1 = (np.sum(data * mat1) / 4.0) - 31.0
print(i1)

mat2 = -mat1 + 1
i2 = (np.sum(data * mat2) / 4.0) - 31.0
print(i2)

Sbc = 4 * np.square(i1) + 4 * np.square(i2)
print(Sbc)

0.5
-0.5
2.0


4.5